# ASO Data Analysis - Task 1: Data Loading & Exploration

This notebook loads, cleans, and explores the ASO ticket and demographic data for FY23 and FY24.

## Overview
- Load ticket data (FY23 and FY24)
- Reshape demographics from long to wide format
- Merge tickets with demographics
- Load event dates and calculate days before event
- Load college pass data
- Generate data quality report
- Save cleaned master dataset


In [17]:
# Import required libraries
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', lambda x: '%.2f' % x)


## 1. Load Ticket Data


In [18]:
# Load FY23 tickets
print("Loading FY23 tickets...")
tickets_fy23 = pd.read_csv('FY23 Regular Tickets.csv')
tickets_fy23['fiscal_year'] = 'FY23'
print(f"FY23 tickets loaded: {len(tickets_fy23):,} records")
print(f"FY23 columns: {tickets_fy23.columns.tolist()}")
print(f"FY23 date range: {tickets_fy23['add_datetime'].min()} to {tickets_fy23['add_datetime'].max()}")
print()

# Load FY24 tickets
print("Loading FY24 tickets...")
tickets_fy24 = pd.read_csv('FY24 Regular Tickets.csv')
tickets_fy24['fiscal_year'] = 'FY24'
print(f"FY24 tickets loaded: {len(tickets_fy24):,} records")
print(f"FY24 columns: {tickets_fy24.columns.tolist()}")
print(f"FY24 date range: {tickets_fy24['add_datetime'].min()} to {tickets_fy24['add_datetime'].max()}")
print()

# Combine into master tickets dataframe
print("Combining tickets...")
master_tickets = pd.concat([tickets_fy23, tickets_fy24], ignore_index=True)
print(f"Master tickets total: {len(master_tickets):,} records")
print(f"Fiscal year distribution:")
print(master_tickets['fiscal_year'].value_counts())
print()

# Display sample
print("Sample of master tickets:")
master_tickets.head()


Loading FY23 tickets...
FY23 tickets loaded: 32,405 records
FY23 columns: ['event_name', 'section_name', 'row_name', 'seat_num', 'last_seat', 'num_seats', 'acct_id', 'zip', 'add_datetime', 'price_code', 'price_code_type', 'price_section', 'comp', 'comp_name', 'purchase_price', 'paid_amount', 'source', 'ticket_type', 'fiscal_year']
FY23 date range: 1/1/2023 to 9/9/2022

Loading FY24 tickets...
FY24 tickets loaded: 35,134 records
FY24 columns: ['event_name', 'section_name', 'row_name', 'seat_num', 'last_seat', 'num_seats', 'acct_id', 'zip', 'add_datetime', 'price_code', 'price_code_type', 'price_section', 'comp', 'comp_name', 'purchase_price', 'paid_amount', 'source', 'ticket_type', 'fiscal_year']
FY24 date range: 1/1/2024 to 9/9/2023

Combining tickets...
Master tickets total: 67,539 records
Fiscal year distribution:
fiscal_year
FY24    35134
FY23    32405
Name: count, dtype: int64

Sample of master tickets:


,event_name,section_name,row_name,seat_num,last_seat,num_seats,acct_id,zip,add_datetime,price_code,price_code_type,price_section,comp,comp_name,purchase_price,paid_amount,source,ticket_type,fiscal_year
0,230922SE,BALCL,A,4,5,2,8362049,30041,7/15/2022,ICO,CYO Subscription,Front Balcony,N,Not Comp,34.00,68.00,online,Subscription,FY23
1,230922SE,BALCL,B,1,2,2,8449640,30316,9/14/2022,IS1,Fixed Sophomore Subscription 6 Concert,Front Balcony,N,Not Comp,23.00,46.00,ASO Staff,Subscription,FY23
2,230922SE,BALCL,B,4,5,2,4061365,30308,7/27/2022,ICO,CYO Subscription,Front Balcony,N,Not Comp,34.00,68.00,online,Subscription,FY23
3,230922SE,BALCL,C,1,2,2,354115,30087,4/5/2022,IE4,Fixed Established Subscription 24 Concert,Front Balcony,N,Not Comp,30.00,60.00,ASO Staff,Subscription,FY23
4,230922SE,BALCL,C,3,4,2,8908470,30253,9/20/2022,I*,Single Ticket,Front Balcony,N,Not Comp,30.00,60.00,online,Single Ticket,FY23


## 2. Load and Reshape Demographics

Demographics files have MULTIPLE ROWS per acct_id:
- Structure: acct_id, attribute_name, attribute_value
- One acct_id will have separate rows for age, gender, etc.
- Need to PIVOT to wide format: one row per acct_id


In [19]:
# Load FY23 demographics
print("Loading FY23 demographics...")
demo_fy23 = pd.read_csv('FY23 Regular Tickets Demographics.csv')
demo_fy23['fiscal_year'] = 'FY23'
print(f"FY23 demographics loaded: {len(demo_fy23):,} records")
print(f"FY23 unique acct_ids: {demo_fy23['acct_id'].nunique():,}")
print(f"FY23 attribute names: {demo_fy23['attrib_name'].unique()}")
print()

# Load FY24 demographics
print("Loading FY24 demographics...")
demo_fy24 = pd.read_csv('FY24 Regular Tickets Demographics.csv')
demo_fy24['fiscal_year'] = 'FY24'
print(f"FY24 demographics loaded: {len(demo_fy24):,} records")
print(f"FY24 unique acct_ids: {demo_fy24['acct_id'].nunique():,}")
print(f"FY24 attribute names: {demo_fy24['attrib_name'].unique()}")
print()

# Show sample of long format
print("Sample of demographics (long format):")
demo_fy23.head(10)


Loading FY23 demographics...
FY23 demographics loaded: 45,494 records
FY23 unique acct_ids: 11,899
FY23 attribute names: ['Input Ind - Education' '1st Ind - Age 2 yr Inc' '1st Ind - Gender'
 'HH Marital Status']

Loading FY24 demographics...
FY24 demographics loaded: 51,857 records
FY24 unique acct_ids: 13,665
FY24 attribute names: ['Input Ind - Education' '1st Ind - Gender' 'HH Marital Status'
 '1st Ind - Age 2 yr Inc']

Sample of demographics (long format):


,acct_id,category,attrib_name,attrib_value,fiscal_year
0,100103,LA Group H: Education Demographics,Input Ind - Education,Graduate School,FY23
1,100103,LA Group A: Gender/Age Demographics,1st Ind - Age 2 yr Inc,36,FY23
2,100103,LA Group A: Gender/Age Demographics,1st Ind - Gender,Male,FY23
3,100103,LA Group B: Household Demographics,HH Marital Status,Married,FY23
4,100378,LA Group B: Household Demographics,HH Marital Status,Married,FY23
5,100378,LA Group A: Gender/Age Demographics,1st Ind - Gender,Female,FY23
6,100378,LA Group A: Gender/Age Demographics,1st Ind - Age 2 yr Inc,82,FY23
7,100378,LA Group H: Education Demographics,Input Ind - Education,Graduate School,FY23
8,100521,LA Group A: Gender/Age Demographics,1st Ind - Gender,Female,FY23
9,100521,LA Group B: Household Demographics,HH Marital Status,Single,FY23


In [20]:
# Combine both demographics files
demo_combined = pd.concat([demo_fy23, demo_fy24], ignore_index=True)
print(f"Combined demographics: {len(demo_combined):,} records")
print(f"Unique acct_ids: {demo_combined['acct_id'].nunique():,}")
print()

# Check for duplicates across fiscal years
acct_ids_fy23 = set(demo_fy23['acct_id'].unique())
acct_ids_fy24 = set(demo_fy24['acct_id'].unique())
overlap = acct_ids_fy23.intersection(acct_ids_fy24)
print(f"Acct_ids appearing in both FY23 and FY24: {len(overlap):,}")
print(f"  (Will keep most recent - FY24 - for these)")
print()

# Sort by fiscal_year (FY24 last) so that when we drop duplicates, we keep FY24
demo_combined = demo_combined.sort_values('fiscal_year')

# Pivot to wide format: one row per acct_id
# First, let's see what attribute names we have
attrib_names = demo_combined['attrib_name'].unique()
print(f"Unique attribute names ({len(attrib_names)}):")
for attr in sorted(attrib_names):
    print(f"  - {attr}")
print()

# Pivot the data
demo_wide = demo_combined.pivot_table(
    index='acct_id',
    columns='attrib_name',
    values='attrib_value',
    aggfunc='first'  # Take first value if duplicates (FY24 will be kept due to sorting)
).reset_index()

# Clean up column names (remove special characters if any)
demo_wide.columns.name = None

# Rename columns to be more readable
column_mapping = {
    '1st Ind - Age 2 yr Inc': 'age',
    '1st Ind - Gender': 'gender',
    'HH Marital Status': 'marital_status',
    'Input Ind - Education': 'education'
}
demo_wide = demo_wide.rename(columns=column_mapping)

print(f"Demographics pivoted to wide format: {len(demo_wide):,} unique acct_ids")
print(f"Columns: {demo_wide.columns.tolist()}")
print()

# Display sample
print("Sample of demographics (wide format):")
demo_wide.head(10)


Combined demographics: 97,351 records
Unique acct_ids: 21,822

Acct_ids appearing in both FY23 and FY24: 3,742
  (Will keep most recent - FY24 - for these)

Unique attribute names (4):
  - 1st Ind - Age 2 yr Inc
  - 1st Ind - Gender
  - HH Marital Status
  - Input Ind - Education

Demographics pivoted to wide format: 21,822 unique acct_ids
Columns: ['acct_id', 'age', 'gender', 'marital_status', 'education']

Sample of demographics (wide format):


,acct_id,age,gender,marital_status,education
0,100103,36,Male,Married,Graduate School
1,100378,82,Female,Married,Graduate School
2,100521,78,Female,Single,Graduate School
3,100532,74,Male,Married,NaN
4,100653,78,Male,Married,College
5,100851,92,Male,Single,Graduate School
6,100928,74,Male,Married,NaN
7,101016,62,Female,Married,High School
8,101126,74,Male,Married,College
9,101742,86,Male,Married,College


In [21]:
# Handle duplicates: if same acct_id appears in both FY23 and FY24, keep most recent
# We already sorted by fiscal_year, so pivot_table with 'first' should keep FY24
# But let's verify by checking if any acct_ids have multiple fiscal years

# Check for any remaining duplicates (shouldn't be any after pivot)
duplicate_accts = demo_wide[demo_wide.duplicated(subset=['acct_id'], keep=False)]
if len(duplicate_accts) > 0:
    print(f"Warning: Found {len(duplicate_accts)} duplicate acct_ids after pivot")
    print(duplicate_accts.head())
    # Remove duplicates, keeping last (FY24)
    demo_wide = demo_wide.drop_duplicates(subset=['acct_id'], keep='last')
    print(f"Removed duplicates. New count: {len(demo_wide):,} unique acct_ids")
else:
    print("✓ No duplicate acct_ids found after pivot")
    print(f"Final demographics: {len(demo_wide):,} unique acct_ids")


✓ No duplicate acct_ids found after pivot
Final demographics: 21,822 unique acct_ids


## 3. Merge Tickets + Demographics


In [22]:
# Left join tickets to demographics on acct_id
print("Merging tickets with demographics...")
master_tickets = master_tickets.merge(
    demo_wide,
    on='acct_id',
    how='left'
)

print(f"After merge: {len(master_tickets):,} records")
print(f"Columns: {len(master_tickets.columns)} total columns")
print()

# Check merge quality
print("Merge quality:")
print(f"  Total tickets: {len(master_tickets):,}")
print(f"  Tickets with acct_id: {master_tickets['acct_id'].notna().sum():,} ({(master_tickets['acct_id'].notna().sum() / len(master_tickets) * 100):.1f}%)")
print(f"  Tickets with demographic data: {master_tickets['age'].notna().sum():,} ({(master_tickets['age'].notna().sum() / len(master_tickets) * 100):.1f}%)")
print()

# Display sample
print("Sample of merged data:")
master_tickets.head()


Merging tickets with demographics...
After merge: 67,539 records
Columns: 23 total columns

Merge quality:
  Total tickets: 67,539
  Tickets with acct_id: 67,539 (100.0%)
  Tickets with demographic data: 60,753 (90.0%)

Sample of merged data:


,event_name,section_name,row_name,seat_num,last_seat,num_seats,acct_id,zip,add_datetime,price_code,price_code_type,price_section,comp,comp_name,purchase_price,paid_amount,source,ticket_type,fiscal_year,age,gender,marital_status,education
0,230922SE,BALCL,A,4,5,2,8362049,30041,7/15/2022,ICO,CYO Subscription,Front Balcony,N,Not Comp,34.00,68.00,online,Subscription,FY23,58,Male,Single,NaN
1,230922SE,BALCL,B,1,2,2,8449640,30316,9/14/2022,IS1,Fixed Sophomore Subscription 6 Concert,Front Balcony,N,Not Comp,23.00,46.00,ASO Staff,Subscription,FY23,80,Male,Married,Graduate School
2,230922SE,BALCL,B,4,5,2,4061365,30308,7/27/2022,ICO,CYO Subscription,Front Balcony,N,Not Comp,34.00,68.00,online,Subscription,FY23,58,Female,Single,College
3,230922SE,BALCL,C,1,2,2,354115,30087,4/5/2022,IE4,Fixed Established Subscription 24 Concert,Front Balcony,N,Not Comp,30.00,60.00,ASO Staff,Subscription,FY23,72,Male,Married,College
4,230922SE,BALCL,C,3,4,2,8908470,30253,9/20/2022,I*,Single Ticket,Front Balcony,N,Not Comp,30.00,60.00,online,Single Ticket,FY23,24,Male,Inferred Single,High School


## 4. Load Event Dates


In [23]:
# STEP 1: Load and examine the events file
print("=" * 80)
print("STEP 1: LOAD EVENTS FILE")
print("=" * 80)
print()

print("Loading events file...")
events = pd.read_excel('23 and 24 Season Events List.xlsx')
print(f"✓ Events file loaded: {len(events):,} records")
print(f"✓ Columns: {events.columns.tolist()}")
print()

# Display structure
print("First 10 rows of events file:")
print(events.head(10))
print()

# Check what columns we have
print("Column details:")
for col in events.columns:
    non_null = events[col].notna().sum()
    print(f"  {col}: {non_null} non-null values")
    if non_null > 0:
        sample_vals = events[col].dropna().head(3).tolist()
        print(f"    Sample values: {sample_vals}")
print()

# Verify we have the columns we need
required_cols = ['Perf Code', 'Date']
missing_cols = [col for col in required_cols if col not in events.columns]
if missing_cols:
    print(f"⚠ WARNING: Missing required columns: {missing_cols}")
else:
    print("✓ All required columns found: 'Perf Code' and 'Date'")


STEP 1: LOAD EVENTS FILE

Loading events file...
✓ Events file loaded: 123 records
✓ Columns: ['Series', 'Weekend', 'Perf Code', 'Name', 'Date', 'Season', 'Type', 'Classical Type']

First 10 rows of events file:
   Series Weekend Perf Code                     Name      Date   Season  \
0     NaN     NaN  23COLLEG     2022/23 College Pass       NaN  2022/23   
1     NaN     NaN  24COLLEG     2023/24 College Pass       NaN  2023/24   
2       1     CS1  230922SE          Opening Weekend   9/22/22  2022/23   
3       1     CS1  230924SE          Opening Weekend   9/24/22  2022/23   
4       2     CS2  231006SE  Beethoven Symphony No 9   10/6/22  2022/23   
5       2     CS2  231008SE  Beethoven Symphony No 9   10/8/22  2022/23   
6  Sunday     CS2  231009S3  Beethoven Symphony No 9   10/9/22  2022/23   
7       3     CS3  231014SE           Lush Romantics  10/14/22  2022/23   
8       3     CS3  231015SE           Lush Romantics  10/15/22  2022/23   
9       4     CS4  231110SE           

### Step 2: Create Events Lookup Table

We'll create a lookup table that maps event codes (like "230922SE") from the "Perf Code" column to their actual event dates from the "Date" column.


In [24]:
# STEP 2: Create events_lookup table
print("=" * 80)
print("STEP 2: CREATE EVENTS LOOKUP TABLE")
print("=" * 80)
print()

# Verify events file is loaded
if 'events' not in locals():
    print("⚠ ERROR: 'events' variable not found. Please run the previous cell first.")
    raise NameError("'events' variable not found")

# Verify required columns exist
if 'Perf Code' not in events.columns:
    print("⚠ ERROR: 'Perf Code' column not found in events file")
    print(f"Available columns: {events.columns.tolist()}")
    raise KeyError("'Perf Code' column not found")

if 'Date' not in events.columns:
    print("⚠ ERROR: 'Date' column not found in events file")
    print(f"Available columns: {events.columns.tolist()}")
    raise KeyError("'Date' column not found")

print("✓ Required columns verified")
print()

# Check for additional columns to include
additional_cols = ['Weekend', 'Series', 'Name', 'Type']
available_additional = [col for col in additional_cols if col in events.columns]
print(f"Additional columns to include: {available_additional}")
print()

# Create events_lookup: Extract Perf Code, Date, and additional columns
print("Creating events_lookup table...")
print("  Using 'Perf Code' column for event codes")
print("  Using 'Date' column for event dates")
if available_additional:
    print(f"  Including additional columns: {available_additional}")
print()

# Select columns to include
cols_to_include = ['Perf Code', 'Date'] + available_additional
events_lookup = events[cols_to_include].copy()

# Rename Perf Code to event_name to match ticket data
events_lookup = events_lookup.rename(columns={'Perf Code': 'event_name'})
# Rename Date to event_date
events_lookup = events_lookup.rename(columns={'Date': 'event_date'})
# Keep other column names as-is (Weekend, Series, Name, Type)

print(f"  Initial rows: {len(events_lookup):,}")
print(f"  Columns: {events_lookup.columns.tolist()}")
print()

# Filter out rows where event_name (Perf Code) is NaN
print("Filtering out rows with NaN event_name...")
before_filter = len(events_lookup)
events_lookup = events_lookup[events_lookup['event_name'].notna()].copy()
after_filter = len(events_lookup)
print(f"  Removed {before_filter - after_filter} rows with NaN event_name")
print(f"  Remaining rows: {after_filter:,}")
print()

# Convert event_date to datetime
print("Converting event_date to datetime...")
events_lookup['event_date'] = pd.to_datetime(events_lookup['event_date'], errors='coerce')
print(f"  Converted dates")
print()

# Remove rows where event_date is still NaN after conversion
print("Filtering out rows with NaN event_date...")
before_date_filter = len(events_lookup)
events_lookup = events_lookup[events_lookup['event_date'].notna()].copy()
after_date_filter = len(events_lookup)
print(f"  Removed {before_date_filter - after_date_filter} rows with NaN event_date")
print(f"  Remaining rows: {after_date_filter:,}")
print()

# Remove duplicates, keeping first occurrence
print("Removing duplicate event_names (keeping first occurrence)...")
before_dedup = len(events_lookup)
events_lookup = events_lookup.drop_duplicates(subset=['event_name'], keep='first')
after_dedup = len(events_lookup)
print(f"  Removed {before_dedup - after_dedup} duplicate event_names")
print(f"  Final unique events: {after_dedup:,}")
print()

# Display summary
print("=" * 80)
print("EVENTS_LOOKUP SUMMARY")
print("=" * 80)
print(f"  Total unique events: {len(events_lookup):,}")
print(f"  Date range: {events_lookup['event_date'].min()} to {events_lookup['event_date'].max()}")
print(f"  Columns: {events_lookup.columns.tolist()}")
print()
print("Sample events_lookup:")
print(events_lookup.head(10))
print()

# Verify structure
print("Verification:")
print(f"  ✓ events_lookup has 'event_name' column: {'event_name' in events_lookup.columns}")
print(f"  ✓ events_lookup has 'event_date' column: {'event_date' in events_lookup.columns}")
if 'Weekend' in events_lookup.columns:
    print(f"  ✓ events_lookup has 'Weekend' column: True")
if 'Series' in events_lookup.columns:
    print(f"  ✓ events_lookup has 'Series' column: True")
if 'Name' in events_lookup.columns:
    print(f"  ✓ events_lookup has 'Name' column: True")
if 'Type' in events_lookup.columns:
    print(f"  ✓ events_lookup has 'Type' column: True")
print(f"  ✓ No NaN event_names: {events_lookup['event_name'].isna().sum() == 0}")
print(f"  ✓ No NaN event_dates: {events_lookup['event_date'].isna().sum() == 0}")
print(f"  ✓ No duplicate event_names: {events_lookup['event_name'].duplicated().sum() == 0}")
print()
print("✓ events_lookup table created successfully!")


STEP 2: CREATE EVENTS LOOKUP TABLE

✓ Required columns verified

Additional columns to include: ['Weekend', 'Series', 'Name', 'Type']

Creating events_lookup table...
  Using 'Perf Code' column for event codes
  Using 'Date' column for event dates
  Including additional columns: ['Weekend', 'Series', 'Name', 'Type']

  Initial rows: 123
  Columns: ['event_name', 'event_date', 'Weekend', 'Series', 'Name', 'Type']

Filtering out rows with NaN event_name...
  Removed 0 rows with NaN event_name
  Remaining rows: 123

Converting event_date to datetime...
  Converted dates

Filtering out rows with NaN event_date...
  Removed 2 rows with NaN event_date
  Remaining rows: 121

Removing duplicate event_names (keeping first occurrence)...
  Removed 0 duplicate event_names
  Final unique events: 121

EVENTS_LOOKUP SUMMARY
  Total unique events: 121
  Date range: 2022-09-22 00:00:00 to 2024-06-16 00:00:00
  Columns: ['event_name', 'event_date', 'Weekend', 'Series', 'Name', 'Type']

Sample events_lo

### Step 3: Verify Event Matching

Before merging, let's check how many ticket event names will match the events_lookup table.


In [25]:
# STEP 3: Verify event matching before merge
print("=" * 80)
print("STEP 3: VERIFY EVENT MATCHING")
print("=" * 80)
print()

# Verify both dataframes exist
if 'master_tickets' not in locals():
    print("⚠ ERROR: 'master_tickets' not found. Please run ticket loading cells first.")
    raise NameError("'master_tickets' not found")

if 'events_lookup' not in locals():
    print("⚠ ERROR: 'events_lookup' not found. Please run the previous cell first.")
    raise NameError("'events_lookup' not found")

# Normalize event names (convert to strings and strip whitespace) for comparison
ticket_events_set = set(str(e).strip() for e in master_tickets['event_name'].unique())
lookup_events_set = set(str(e).strip() for e in events_lookup['event_name'].unique())

# Check matching
matching_events = ticket_events_set.intersection(lookup_events_set)
missing_events = ticket_events_set - lookup_events_set

print(f"Ticket event analysis:")
print(f"  Unique event names in tickets: {len(ticket_events_set)}")
print(f"  Sample ticket events: {sorted(list(ticket_events_set))[:10]}")
print()

print(f"Events lookup analysis:")
print(f"  Unique event names in events_lookup: {len(lookup_events_set)}")
print(f"  Sample lookup events: {sorted(list(lookup_events_set))[:10]}")
print()

print(f"Matching results:")
print(f"  ✓ Matching events: {len(matching_events)}")
match_pct = (len(matching_events) / len(ticket_events_set) * 100) if len(ticket_events_set) > 0 else 0
print(f"  Match rate: {match_pct:.1f}%")
print(f"  ⚠ Missing from events file: {len(missing_events)}")
print()

if len(matching_events) > 0:
    print(f"  Sample matched events: {sorted(list(matching_events))[:10]}")
    print()

if len(missing_events) > 0:
    print(f"  ⚠ Events in tickets but NOT in events file:")
    if len(missing_events) <= 20:
        print(f"    {sorted(list(missing_events))}")
    else:
        print(f"    First 20: {sorted(list(missing_events))[:20]}")
        print(f"    (Total missing: {len(missing_events)})")
    print()

# Check if events_lookup has the right structure
print("events_lookup structure check:")
print(f"  Columns: {events_lookup.columns.tolist()}")
print(f"  Has 'event_name': {'event_name' in events_lookup.columns}")
print(f"  Has 'event_date': {'event_date' in events_lookup.columns}")
print()

if len(matching_events) == 0:
    print("⚠ WARNING: No matching events found!")
    print("  This means the merge will result in all NULL event_date values.")
    print("  Please check the sample outputs above for differences")
else:
    print(f"✓ Ready to merge! {len(matching_events)} events will be matched.")


STEP 3: VERIFY EVENT MATCHING

Ticket event analysis:
  Unique event names in tickets: 121
  Sample ticket events: ['230104SE', '230112SE', '230115S3', '230119SE', '230121SE', '230126SE', '230128SE', '230202SE', '230203SE', '230209SE']

Events lookup analysis:
  Unique event names in events_lookup: 121
  Sample lookup events: ['230104SE', '230112SE', '230115S3', '230119SE', '230121SE', '230126SE', '230128SE', '230202SE', '230203SE', '230209SE']

Matching results:
  ✓ Matching events: 120
  Match rate: 99.2%
  ⚠ Missing from events file: 1

  Sample matched events: ['230104SE', '230112SE', '230115S3', '230119SE', '230121SE', '230126SE', '230128SE', '230202SE', '230203SE', '230209SE']

  ⚠ Events in tickets but NOT in events file:
    ['231218SE']

events_lookup structure check:
  Columns: ['event_name', 'event_date', 'Weekend', 'Series', 'Name', 'Type']
  Has 'event_name': True
  Has 'event_date': True

✓ Ready to merge! 120 events will be matched.


### Step 4: Merge Event Dates

Now we'll merge the event dates into master_tickets by matching `event_name` (from tickets) with `event_name` (from events_lookup, which came from "Perf Code").


In [26]:
# STEP 4: Merge event dates into master_tickets
print("=" * 80)
print("STEP 4: MERGE EVENT DATES")
print("=" * 80)
print()

# Verify prerequisites
if 'master_tickets' not in locals():
    print("⚠ ERROR: 'master_tickets' not found. Please run ticket loading cells first.")
    raise NameError("'master_tickets' not found")

if 'events_lookup' not in locals():
    print("⚠ ERROR: 'events_lookup' not found. Please run previous cells first.")
    raise NameError("'events_lookup' not found")

# Verify events_lookup structure
if 'event_name' not in events_lookup.columns:
    print("⚠ ERROR: events_lookup missing 'event_name' column")
    print(f"  Available columns: {events_lookup.columns.tolist()}")
    raise KeyError("'event_name' column not found in events_lookup")

if 'event_date' not in events_lookup.columns:
    print("⚠ ERROR: events_lookup missing 'event_date' column")
    print(f"  Available columns: {events_lookup.columns.tolist()}")
    raise KeyError("'event_date' column not found in events_lookup")

print("✓ Prerequisites verified")
print(f"  master_tickets: {len(master_tickets):,} records")
print(f"  events_lookup: {len(events_lookup):,} records")
print(f"  events_lookup columns: {events_lookup.columns.tolist()}")
print()

# Normalize event names before merging (strip whitespace and ensure string type)
print("Normalizing event names before merge...")
master_tickets['event_name'] = master_tickets['event_name'].astype(str).str.strip()
events_lookup['event_name'] = events_lookup['event_name'].astype(str).str.strip()
print("  ✓ Stripped whitespace and converted to strings")
print()

# Check matching after normalization
unique_ticket_events = set(master_tickets['event_name'].unique())
unique_events_lookup = set(events_lookup['event_name'].unique())
matching_events = unique_ticket_events.intersection(unique_events_lookup)

print(f"Pre-merge check:")
print(f"  Unique ticket events: {len(unique_ticket_events)}")
print(f"  Unique lookup events: {len(unique_events_lookup)}")
print(f"  Events that will match: {len(matching_events)}")
if len(matching_events) > 0:
    print(f"  Sample matches: {sorted(list(matching_events))[:5]}")
print()

# Perform merge
print("Merging event dates and additional event info...")
print(f"  Merging on column: 'event_name'")
print(f"  Merge type: left join (keep all tickets)")
additional_cols = [col for col in events_lookup.columns if col not in ['event_name', 'event_date']]
if additional_cols:
    print(f"  Additional columns to merge: {additional_cols}")
print()

master_tickets = master_tickets.merge(
    events_lookup,
    on='event_name',
    how='left'
)

print(f"✓ Merge completed")
print(f"  Records after merge: {len(master_tickets):,}")
print(f"  New columns added: {len(events_lookup.columns) - 1} (event_name used for merge)")  # -1 because event_name is the merge key
print()

# Check merge results
matched_count = master_tickets['event_date'].notna().sum()
matched_pct = (matched_count / len(master_tickets) * 100) if len(master_tickets) > 0 else 0
unmatched_count = master_tickets['event_date'].isna().sum()

print("=" * 80)
print("MERGE RESULTS")
print("=" * 80)
print(f"  Tickets with event_date: {matched_count:,} ({matched_pct:.1f}%)")
print(f"  Tickets without event_date: {unmatched_count:,} ({100-matched_pct:.1f}%)")
print()

# Show coverage of additional columns if they exist
if additional_cols:
    print("Additional event info coverage:")
    for col in additional_cols:
        if col in master_tickets.columns:
            non_null = master_tickets[col].notna().sum()
            pct = (non_null / len(master_tickets) * 100) if len(master_tickets) > 0 else 0
            print(f"  {col}: {non_null:,} tickets ({pct:.1f}%)")
    print()

if matched_count == 0:
    print("⚠ WARNING: No matches found! All event_date values are NULL.")
    print("  This suggests:")
    print("    1. Event names still don't match after normalization")
    print("    2. Please check the verification step above")
    print("    3. Check if events_lookup was created correctly")
else:
    print(f"✓ Successfully matched {matched_count:,} tickets with event dates")
    print()

# Calculate days_before_event for matched tickets
print("Calculating days_before_event...")

# Ensure add_datetime is datetime
if 'add_datetime' in master_tickets.columns:
    master_tickets['add_datetime'] = pd.to_datetime(master_tickets['add_datetime'], errors='coerce')
    print("  ✓ Converted add_datetime to datetime")
else:
    print("  ⚠ WARNING: 'add_datetime' column not found")

# Calculate days_before_event: (event_date - add_datetime)
master_tickets['days_before_event'] = (
    master_tickets['event_date'] - master_tickets['add_datetime']
).dt.days

valid_days = master_tickets['days_before_event'].dropna()
print(f"  Calculated days_before_event for {len(valid_days):,} tickets")
print()

if len(valid_days) > 0:
    print("days_before_event statistics:")
    print(f"  Mean: {valid_days.mean():.1f} days")
    print(f"  Median: {valid_days.median():.1f} days")
    print(f"  Min: {valid_days.min():.0f} days")
    print(f"  Max: {valid_days.max():.0f} days")
    print()
    
    # Show breakdown
    print("Purchase timing breakdown:")
    early = (valid_days >= 30).sum()
    moderate = ((valid_days >= 7) & (valid_days < 30)).sum()
    last_minute = (valid_days < 7).sum()
    print(f"  Early (30+ days): {early:,} tickets")
    print(f"  Moderate (7-29 days): {moderate:,} tickets")
    print(f"  Last minute (<7 days): {last_minute:,} tickets")
else:
    print("  ⚠ No valid days_before_event calculated (all values are NaN)")
    print()

# Show unmatched events (if any)
if unmatched_count > 0:
    print("Unmatched events (tickets without event_date):")
    unmatched_events = master_tickets[master_tickets['event_date'].isna()]['event_name'].value_counts().head(10)
    for event, count in unmatched_events.items():
        print(f"  {event}: {count:,} tickets")
    print()

# Display sample
print("Sample results:")
sample_cols = ['event_name', 'add_datetime', 'event_date', 'days_before_event'] + additional_cols
available_cols = [col for col in sample_cols if col in master_tickets.columns]
master_tickets[available_cols].head(10)


STEP 4: MERGE EVENT DATES

✓ Prerequisites verified
  master_tickets: 67,539 records
  events_lookup: 121 records
  events_lookup columns: ['event_name', 'event_date', 'Weekend', 'Series', 'Name', 'Type']

Normalizing event names before merge...
  ✓ Stripped whitespace and converted to strings

Pre-merge check:
  Unique ticket events: 121
  Unique lookup events: 121
  Events that will match: 120
  Sample matches: ['230104SE', '230112SE', '230115S3', '230119SE', '230121SE']

Merging event dates and additional event info...
  Merging on column: 'event_name'
  Merge type: left join (keep all tickets)
  Additional columns to merge: ['Weekend', 'Series', 'Name', 'Type']

✓ Merge completed
  Records after merge: 67,539
  New columns added: 5 (event_name used for merge)

MERGE RESULTS
  Tickets with event_date: 66,961 (99.1%)
  Tickets without event_date: 578 (0.9%)

Additional event info coverage:
  Weekend: 66,961 tickets (99.1%)
  Series: 66,961 tickets (99.1%)
  Name: 66,961 tickets (99.1

,event_name,add_datetime,event_date,days_before_event,Weekend,Series,Name,Type
0,230922SE,2022-07-15,2022-09-22,69.00,CS1,1,Opening Weekend,Classical
1,230922SE,2022-09-14,2022-09-22,8.00,CS1,1,Opening Weekend,Classical
2,230922SE,2022-07-27,2022-09-22,57.00,CS1,1,Opening Weekend,Classical
3,230922SE,2022-04-05,2022-09-22,170.00,CS1,1,Opening Weekend,Classical
4,230922SE,2022-09-20,2022-09-22,2.00,CS1,1,Opening Weekend,Classical
5,230922SE,2022-08-30,2022-09-22,23.00,CS1,1,Opening Weekend,Classical
6,230922SE,2022-04-08,2022-09-22,167.00,CS1,1,Opening Weekend,Classical
7,230922SE,2022-04-22,2022-09-22,153.00,CS1,1,Opening Weekend,Classical
8,230922SE,2022-09-16,2022-09-22,6.00,CS1,1,Opening Weekend,Classical
9,230922SE,2022-09-07,2022-09-22,15.00,CS1,1,Opening Weekend,Classical


## Create Derived Variables

In [91]:
#create derived variables for month, month_name and season_period
master_tickets['month'] = master_tickets['add_datetime'].dt.month
master_tickets['month_name'] = master_tickets['add_datetime'].dt.month_name()

# Define season periods
#  * season_period = categorize by month:
#     - "Opening" for September-October
#     - "Mid-Season" for November-February  
#     - "Peak" for March-May
#     - "Finale" for June
def assign_season_period(month):
    if month in [9, 10]:
        return 'Opening'
    elif month in [11, 12, 1, 2]:
        return 'Mid-Season'
    elif month in [3, 4, 5]:
        return 'Peak'
    elif month == 6:
        return 'Finale'
    else:
        return 'Off-Season'
    

master_tickets['season_period'] = master_tickets['month'].apply(assign_season_period)
print("Derived variables created: month, month_name, season_period")
print("Sample of derived variables:")
master_tickets[['add_datetime', 'month', 'month_name', 'season_period']].head(10)



Derived variables created: month, month_name, season_period
Sample of derived variables:


,add_datetime,month,month_name,season_period
0,2022-07-15,7,July,Off-Season
1,2022-09-14,9,September,Opening
2,2022-07-27,7,July,Off-Season
3,2022-04-05,4,April,Peak
4,2022-09-20,9,September,Opening
5,2022-08-30,8,August,Off-Season
6,2022-04-08,4,April,Peak
7,2022-04-22,4,April,Peak
8,2022-09-16,9,September,Opening
9,2022-09-07,9,September,Opening


## 5. Load College Pass Data


In [92]:
# Load college pass tickets
print("Loading college pass tickets...")
college_pass_tickets = pd.read_csv('College Pass Concert Tickets.csv')
college_pass_tickets['fiscal_year'] = 'College Pass'
college_pass_tickets['is_college_pass'] = True
print(f"College pass tickets loaded: {len(college_pass_tickets):,} records")
print(f"Columns: {college_pass_tickets.columns.tolist()}")
print()

# Display sample
print("Sample of college pass tickets:")
college_pass_tickets.head(10)


Loading college pass tickets...
College pass tickets loaded: 5,995 records
Columns: ['event_name', 'section_name', 'row_name', 'seat_num', 'last_seat', 'num_seats', 'acct_id', 'zip', 'add_datetime', 'price_code', 'comp', 'comp_name', 'purchase_price', 'paid_amount', 'source', 'fiscal_year', 'is_college_pass']

Sample of college pass tickets:


,event_name,section_name,row_name,seat_num,last_seat,num_seats,acct_id,zip,add_datetime,price_code,comp,comp_name,purchase_price,paid_amount,source,fiscal_year,is_college_pass
0,240420C1,GA1,G01,11,13,3,9176172,30084,3/31/2024,A0B,N,Not Comp,10,30,online,College Pass,True
1,240419BT,GA2,G08,29,29,1,8943006,30363,4/16/2024,A0E,N,Not Comp,0,0,online,College Pass,True
2,240419BT,GA2,G08,40,40,1,9617850,30083,4/16/2024,A0E,N,Not Comp,0,0,online,College Pass,True
3,240419BT,GA2,G08,44,44,1,8942894,20176,4/16/2024,A0E,N,Not Comp,0,0,online,College Pass,True
4,240419BT,GA2,G09,11,11,1,9747444,30075,4/16/2024,A0E,N,Not Comp,0,0,online,College Pass,True
5,240419BT,GA2,G09,35,35,1,8630517,30032,4/16/2024,A0E,N,Not Comp,0,0,online,College Pass,True
6,240419BT,GA2,G09,40,40,1,8784081,30309,4/16/2024,A0E,N,Not Comp,0,0,online,College Pass,True
7,240419BT,GA2,G09,41,41,1,8991929,19341,4/16/2024,A0E,N,Not Comp,0,0,online,College Pass,True
8,240419BT,GA2,G10,9,9,1,9531691,30306,4/16/2024,A0E,N,Not Comp,0,0,online,College Pass,True
9,240419BT,GA2,G10,24,24,1,9594082,30032,4/16/2024,A0E,N,Not Comp,0,0,online,College Pass,True


In [93]:
# Load college pass demographics
print("Loading college pass demographics...")
college_pass_demo = pd.read_csv('College Pass Demographics.csv')
print(f"College pass demographics loaded: {len(college_pass_demo):,} records")
print(f"Unique acct_ids: {college_pass_demo['acct_id'].nunique():,}")
print(f"Attribute names: {college_pass_demo['attrib_name'].unique()}")
print()

# Reshape college pass demographics same way as above
college_pass_demo_wide = college_pass_demo.pivot_table(
    index='acct_id',
    columns='attrib_name',
    values='attrib_value',
    aggfunc='first'
).reset_index()

college_pass_demo_wide.columns.name = None

# Rename columns to match
column_mapping = {
    '1st Ind - Age 2 yr Inc': 'age',
    '1st Ind - Gender': 'gender',
    'HH Marital Status': 'marital_status',
    'Input Ind - Education': 'education'
}
college_pass_demo_wide = college_pass_demo_wide.rename(columns=column_mapping)

print(f"College pass demographics pivoted: {len(college_pass_demo_wide):,} unique acct_ids")
print()

# Merge college pass tickets with demographics
college_pass_tickets = college_pass_tickets.merge(
    college_pass_demo_wide,
    on='acct_id',
    how='left'
)

print(f"College pass tickets merged with demographics: {len(college_pass_tickets):,} records")
print()

# Merge event dates for college pass tickets
if 'event_name' in college_pass_tickets.columns:
    college_pass_tickets = college_pass_tickets.merge(
        events_lookup,
        on='event_name',
        how='left'
    )
    
    # Convert add_datetime and calculate days_before_event
    college_pass_tickets['add_datetime'] = pd.to_datetime(college_pass_tickets['add_datetime'], errors='coerce')
    college_pass_tickets['days_before_event'] = (
        college_pass_tickets['event_date'] - college_pass_tickets['add_datetime']
    ).dt.days
    
    print("College pass tickets merged with event dates")

# Ensure regular tickets have is_college_pass flag
if 'is_college_pass' not in master_tickets.columns:
    master_tickets['is_college_pass'] = False

print("\nCollege pass data prepared")
print(f"College pass tickets: {len(college_pass_tickets):,} records")
print(f"Regular tickets: {len(master_tickets):,} records")


Loading college pass demographics...
College pass demographics loaded: 2,727 records
Unique acct_ids: 745
Attribute names: ['1st Ind - Age 2 yr Inc' '1st Ind - Gender' 'HH Marital Status'
 'Input Ind - Education']

College pass demographics pivoted: 745 unique acct_ids

College pass tickets merged with demographics: 5,995 records

College pass tickets merged with event dates

College pass data prepared
College pass tickets: 5,995 records
Regular tickets: 67,539 records


## 6. Data Quality Report


In [94]:
print("=" * 80)
print("DATA QUALITY REPORT")
print("=" * 80)
print()

# 1. Total ticket records in FY23 vs FY24
print("1. TICKET RECORDS BY FISCAL YEAR")
print("-" * 80)
fy_counts = master_tickets['fiscal_year'].value_counts().sort_index()
for fy, count in fy_counts.items():
    print(f"  {fy}: {count:,} records")
print(f"  Total: {len(master_tickets):,} records")
print()

# 2. Unique acct_ids in FY23 vs FY24
print("2. UNIQUE ACCOUNT IDs BY FISCAL YEAR")
print("-" * 80)
for fy in ['FY23', 'FY24']:
    fy_data = master_tickets[master_tickets['fiscal_year'] == fy]
    unique_accts = fy_data['acct_id'].nunique()
    total_records = len(fy_data)
    print(f"  {fy}: {unique_accts:,} unique acct_ids (from {total_records:,} records)")
print(f"  Combined: {master_tickets['acct_id'].nunique():,} unique acct_ids")
print()

# 3. Missing values
print("3. MISSING VALUES")
print("-" * 80)
total_records = len(master_tickets)

# acct_id nulls
acct_id_nulls = master_tickets['acct_id'].isna().sum()
acct_id_pct = (acct_id_nulls / total_records) * 100
print(f"  acct_id nulls: {acct_id_nulls:,} ({acct_id_pct:.2f}%)")

# age nulls
age_nulls = master_tickets['age'].isna().sum()
age_pct = (age_nulls / total_records) * 100
print(f"  age nulls: {age_nulls:,} ({age_pct:.2f}%)")

# Calculate % of patrons (unique acct_ids) with age data
unique_accts_with_age = master_tickets[master_tickets['age'].notna()]['acct_id'].nunique()
unique_accts_total = master_tickets['acct_id'].nunique()
patrons_with_age_pct = (unique_accts_with_age / unique_accts_total) * 100 if unique_accts_total > 0 else 0
print(f"  Patrons with age data: {unique_accts_with_age:,} of {unique_accts_total:,} ({patrons_with_age_pct:.2f}%)")

# event_date nulls
event_date_nulls = master_tickets['event_date'].isna().sum()
event_date_pct = (event_date_nulls / total_records) * 100
print(f"  event_date nulls: {event_date_nulls:,} ({event_date_pct:.2f}%)")

# Other demographic fields
print("\n  Other demographic fields:")
demo_fields = ['gender', 'marital_status', 'education']
for field in demo_fields:
    if field in master_tickets.columns:
        nulls = master_tickets[field].isna().sum()
        pct = (nulls / total_records) * 100
        print(f"    {field} nulls: {nulls:,} ({pct:.2f}%)")
print()

# 4. Date range validation
print("4. DATE RANGE VALIDATION")
print("-" * 80)
master_tickets['add_datetime'] = pd.to_datetime(master_tickets['add_datetime'], errors='coerce')

for fy in ['FY23', 'FY24']:
    fy_data = master_tickets[master_tickets['fiscal_year'] == fy]
    min_date = fy_data['add_datetime'].min()
    max_date = fy_data['add_datetime'].max()
    print(f"  {fy}:")
    print(f"    Date range: {min_date} to {max_date}")
    
    # Expected ranges
    if fy == 'FY23':
        expected_start = pd.to_datetime('2022-09-01')
        expected_end = pd.to_datetime('2023-06-30')
        print(f"    Expected: Sept 2022 - June 2023")
    else:
        expected_start = pd.to_datetime('2023-09-01')
        expected_end = pd.to_datetime('2024-06-30')
        print(f"    Expected: Sept 2023 - June 2024")
    
    # Check if dates are within expected range
    if min_date < expected_start or max_date > expected_end:
        print(f"    ⚠ Warning: Some dates outside expected range")
    else:
        print(f"    ✓ Dates within expected range")
print()

# 5. Comp ticket count and % of total
print("5. COMP TICKET ANALYSIS")
print("-" * 80)
if 'comp' in master_tickets.columns:
    comp_counts = master_tickets['comp'].value_counts()
    total_comps = master_tickets['comp'].eq('Y').sum() if 'comp' in master_tickets.columns else 0
    comp_pct = (total_comps / total_records) * 100
    print(f"  Comp tickets: {total_comps:,} ({comp_pct:.2f}%)")
    print(f"  Non-comp tickets: {total_records - total_comps:,} ({100 - comp_pct:.2f}%)")
    print(f"  Distribution:")
    for val, count in comp_counts.items():
        print(f"    {val}: {count:,} ({count/total_records*100:.2f}%)")
else:
    print("  'comp' column not found")
print()

# 6. Price distribution
print("6. PRICE DISTRIBUTION")
print("-" * 80)
if 'paid_amount' in master_tickets.columns:
    paid_amount = master_tickets['paid_amount'].dropna()
    print(f"  Min: ${paid_amount.min():.2f}")
    print(f"  Max: ${paid_amount.max():.2f}")
    print(f"  Mean: ${paid_amount.mean():.2f}")
    print(f"  Median: ${paid_amount.median():.2f}")
    print(f"  Std Dev: ${paid_amount.std():.2f}")
    print(f"\n  Percentiles:")
    for p in [10, 25, 50, 75, 90, 95, 99]:
        val = paid_amount.quantile(p/100)
        print(f"    {p}th: ${val:.2f}")
else:
    print("  'paid_amount' column not found")
print()

# 7. Demographics coverage
print("7. DEMOGRAPHICS COVERAGE")
print("-" * 80)
print("  Available demographic attributes:")
demo_cols = [col for col in master_tickets.columns if col in ['age', 'gender', 'marital_status', 'education']]
for col in demo_cols:
    non_null = master_tickets[col].notna().sum()
    pct = (non_null / total_records) * 100
    unique_vals = master_tickets[col].nunique()
    print(f"    {col}:")
    print(f"      Coverage: {non_null:,} records ({pct:.2f}%)")
    print(f"      Unique values: {unique_vals}")
    if unique_vals <= 20:
        print(f"      Values: {master_tickets[col].value_counts().head(10).to_dict()}")

# Check for other demographic attributes from original pivot
other_demo_cols = [col for col in master_tickets.columns 
                   if col not in ['acct_id', 'fiscal_year', 'is_college_pass'] 
                   and col not in demo_cols
                   and master_tickets[col].dtype == 'object'
                   and master_tickets[col].notna().sum() > 0]

if other_demo_cols:
    print("\n  Other demographic-like attributes found:")
    for col in other_demo_cols[:10]:  # Limit to first 10
        non_null = master_tickets[col].notna().sum()
        pct = (non_null / total_records) * 100
        print(f"    {col}: {non_null:,} records ({pct:.2f}%)")

print()
print("=" * 80)


DATA QUALITY REPORT

1. TICKET RECORDS BY FISCAL YEAR
--------------------------------------------------------------------------------
  FY23: 32,405 records
  FY24: 35,134 records
  Total: 67,539 records

2. UNIQUE ACCOUNT IDs BY FISCAL YEAR
--------------------------------------------------------------------------------
  FY23: 13,093 unique acct_ids (from 32,405 records)
  FY24: 15,363 unique acct_ids (from 35,134 records)
  Combined: 24,603 unique acct_ids

3. MISSING VALUES
--------------------------------------------------------------------------------
  acct_id nulls: 0 (0.00%)
  age nulls: 6,786 (10.05%)
  Patrons with age data: 20,864 of 24,603 (84.80%)
  event_date nulls: 67,539 (100.00%)

  Other demographic fields:
    gender nulls: 4,812 (7.12%)
    marital_status nulls: 5,338 (7.90%)
    education nulls: 13,905 (20.59%)

4. DATE RANGE VALIDATION
--------------------------------------------------------------------------------
  FY23:
    Date range: 2022-03-02 00:00:00 to 

In [95]:
# Prepare final dataset for saving
print("Preparing master dataset for saving...")

# Select and order columns logically
column_order = [
    # Identifiers
    'fiscal_year',
    'acct_id',
    'is_college_pass',
    # Event info
    'event_name',
    'event_date',
    'add_datetime',
    'days_before_event',
    # Seating
    'section_name',
    'row_name',
    'seat_num',
    'last_seat',
    'num_seats',
    # Pricing
    'price_code',
    'price_code_type',
    'price_section',
    'purchase_price',
    'paid_amount',
    'comp',
    'comp_name',
    'source',
    'ticket_type',
    # Demographics
    'age',
    'gender',
    'marital_status',
    'education',
    # Location
    'zip'
]

# Add any additional columns that exist
existing_cols = [col for col in column_order if col in master_tickets.columns]
other_cols = [col for col in master_tickets.columns if col not in column_order]
final_cols = existing_cols + other_cols

master_tickets_clean = master_tickets[final_cols].copy()

print(f"Master dataset prepared: {len(master_tickets_clean):,} records, {len(master_tickets_clean.columns)} columns")
print(f"Columns: {master_tickets_clean.columns.tolist()}")
print()

# Display summary
print("Summary:")
print(master_tickets_clean.info())
print()

# Save to CSV
output_file = 'master_tickets.csv'
print(f"Saving to {output_file}...")
master_tickets_clean.to_csv(output_file, index=False)
print(f"✓ Saved successfully!")
print(f"  File: {output_file}")
print(f"  Records: {len(master_tickets_clean):,}")
print(f"  Columns: {len(master_tickets_clean.columns)}")


Preparing master dataset for saving...
Master dataset prepared: 67,539 records, 29 columns
Columns: ['fiscal_year', 'acct_id', 'is_college_pass', 'event_name', 'event_date', 'add_datetime', 'days_before_event', 'section_name', 'row_name', 'seat_num', 'last_seat', 'num_seats', 'price_code', 'price_code_type', 'price_section', 'purchase_price', 'paid_amount', 'comp', 'comp_name', 'source', 'ticket_type', 'age', 'gender', 'marital_status', 'education', 'zip', 'month', 'month_name', 'season_period']

Summary:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67539 entries, 0 to 67538
Data columns (total 29 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   fiscal_year        67539 non-null  object        
 1   acct_id            67539 non-null  int64         
 2   is_college_pass    67539 non-null  bool          
 3   event_name         67539 non-null  object        
 4   event_date         0 non-null      datetime64

## Summary

This notebook has completed the following tasks:

1. ✅ Loaded FY23 and FY24 ticket data
2. ✅ Loaded and reshaped demographics from long to wide format
3. ✅ Merged tickets with demographics
4. ✅ Loaded event dates and calculated days_before_event
5. ✅ Loaded college pass data
6. ✅ Generated comprehensive data quality report
7. ✅ Saved cleaned master dataset as `master_tickets.csv`

